<h1><center>EXTRAÇÃO, TRATAMENTO E MODELAGEM DOS DADOS DA BOLSA DE VALORES BRASILEIRA

<P>O objetivo deste artigo é ensinar passo a passo como extrair e modelar os dados da bolsa de valores, utilizando a linguagem de programação Python e a biblioteca Pandas. Então, o primeiro passo é entender que tipo de dados estamos procurando. Porque, dentro do site da B3, exitem diversos DataSets para análise.
Sugiro pesquisar no site oficial as opções disponíveis: https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/
Aqui, utilizaremos os dados do último mês disponível (07/2022) e posteriormente vamos automatizar o código que faremos hoje, para que ele seja atualizado automaticamente.

<img src='txt_img_0722.jpg'></img>

Temos então, um arquivo txt com dados semiestruturados pois, eles não possuem uma estrutura clara porém, possuem delimitações que permitirão a estruturação dos dados para utilização por exemplo, no Power BI ou Excell.
Lembrando que este é um estudo da linguagem Python, vamos começar importando a biblioteca Pandas, que será a única utilizada neste artigo.
Não se esqueça de instalar a biblioteca dentro do seu editor de códigos.


In [ ]:
import pandas as pd

Antes de importar os dados, precisamos saber quais dados nós estamos procurando e quais dados estão disponíveis no arquivo TXT. Para isso, existe uma documentação chamada de Layout cujas informações nortearão nossa busca. No caso da Bovespa, existe um arquivo pdf disponível na página de históricos.
Para este artigo, utilizaremos:
- A data do Pregão;
- O código da ação;
- O nome da empresa;
- O preço de abertura das ações;
- O preço máximo negociado;
- O preço mínimo negociado;
- O preço de fechamento das ações;
- A quantidade de negócios;
- O volume financeiro negociado;

Também, no caso das ações da Bovespa e para o nosso estudo, precisamos entender que os dados relevantes serão as negociações por lote padrão. Entretanto, poderíamos por exemplo, buscar também as negociações no mercado fracionário afim de estudar correlações entre os mercados. Por fim, entendemos que o arquivo da Bovespa trata-se de um arquivo de dados em formato de largura fixa então, neste artigo iremos uilizar a função read.fwf para importar os dados que vamos utilizar.
Para começar, vamos criar duas listas, uma contendo os valores da largura do texto do nosso DataSet e outra com os nomes das variáveis que deverão receber esses valores.




In [ ]:
table = [(2,10), (10,12), (12,24), (27,39), 
(56,69),(69,82),(82,95),
(108,121),(152,170),(170,188)]

name_table = ['data_pregao','cod_bdi', 'cod_acao', 'nome_emp', 'preco_abertura', 'preco_maximo', 'preco_minimo', 
'preco_fechamento', 'qtd_negocios', 'vol_financeiro']


Importante! As informações da variável 'table' foram fornecidas pelo pdf com as informações de layout citadas anteriormente neste artigo.

Finalmente, podemos aplicar o método read fwf para importar os dados necessários e criar o nosso DataFrame.

In [ ]:

df_table = pd.read_fwf("./Arquivos/hist_ult12meses_a_vista_072022.TXT", colspecs = table,
names = name_table, skiprows=1)
display(df_table)


Já temos o nosso primeiro DataBase, podemos notar que ainda não é uma tabela pronta para usarmos em análises pois, existe ainda muita informação não filtradas. Então, nesta primeira parte da limpeza dos dados, vamos filtrar o 'cod_bdi' que, conforme explicado no arquivo de layout, traz o filtro de mercado à vista que são os valores que queremos analisar.

In [ ]:
df_table = df_table[df_table['cod_bdi'] == 2]
display(df_table)

Agora, temos a tabela com os valores que desejamos utilizar. Portanto, vamos fazer alguns tratamentos para limpar os dados e corrigir os tipos dos valores importados. Porque, apesar dos valores já estarem organizados em colunas, a formatação ainda está errada então, vamos transformar os valores de data em data, excluir o cod_bdi que já utilizamos e transformar os números para a tipagem correta.

In [ ]:
df_table = df_table.drop(['cod_bdi'], axis=1)
df_table['data_pregao'] = pd.to_datetime(df_table['data_pregao'], format='%Y%m%d')
df_table['preco_abertura'] = (df_table['preco_abertura']/100).astype(float)
df_table['preco_maximo'] = (df_table['preco_maximo']/100).astype(float)
df_table['preco_minimo'] = (df_table['preco_minimo']/100).astype(float)
df_table['preco_fechamento'] = (df_table['preco_fechamento']/100).astype(float)
df_table['qtd_negocios'] = df_table['qtd_negocios'].astype(int)
df_table['vol_financeiro'] = df_table['vol_financeiro'].astype(int)
df_table['nome_emp'] = df_table['nome_emp'].astype(str)


Concluímos a primeira parte da nossa tarefa de extração dos dados.
Vou criar uma nova variável, contendo as informações desta tabela para continuar com a limpeza dos dados. Faço isso, para o caso de precisar verificar a tabela inicial em algum momento porque assim, sempre tenho as primeiras informações disponíveis.

In [ ]:
df = df_table
display(df)

A primeira coisa que é importante fazer, é analisar as primeiras possibilidades de erros nas informações e para isso, vamos fazer algumas análises gerais dos valores da tabela como, número de colunas e linhas, informações gerais, se existem valores nulos e possíveis discrepâncias.

In [ ]:
#display(df.shape)
#df.isnull().sum()
df.describe()

Já sabemos agora, que o número de linhas é de 17607, não há valores nulos e na descrição existe uma informação importante para verificarmos. No 'max' que mostra os valores máximos de cada coluna, temos uma ação que foi negociada a R$98.335,00 enquanto, a média está em torno de R$100,00.
Vamos verificar este valor.

In [ ]:
df.sort_values(by =['preco_abertura'], ascending=False)

Como imaginado, o valor discrepante é referente ao IBOV, que é um dado importante porém, não é o foco do nosso estudo. Por isso, vamos excluir a linha que consta o IBOV11 também, podemos ver que existem ações que não tiveram negocios no mercado à vista então, vamos excluir aas linhas com valores de 'qtd_negocios' zerados.

In [ ]:
df.drop([19087],inplace=True)
no_exchange_df = df[df['qtd_negocios'] == 0]
df = df.drop(no_exchange_df.index)
df.describe()

Temos então, uma tabela em que todas as ações tiveram alguma operação no período.
A partir daqui, é importante termos uma boa ideia do que queremos analisar para melhorar ainda mais os dados portanto, vou adicionar duas colunas com o valor percentual de variação das ações, uma para os valores de abertura e fechamento e outro, para os valores máximo e mínimo.

In [ ]:
df['perc_fechamento'] = ((df['preco_fechamento'] / df['preco_abertura'])-1)*100
df['perc_max'] = ((df['preco_minimo'] / df['preco_maximo'])-1)*(-100)
display(df)

Uma vez que temos esses novos dados, podemos fazer uma análise que eu julgo interessante. Quantas ações e dias a variação diária de ações que tiveram negócios foi zero ou seja, quando fecharam no mesmo valor de abertura.
Vamos salvar este filtro e retirar as ações que tiveram este comportamento para continuar nossos estudos.

In [ ]:
zero_a_zero_df = df[df['perc_fechamento']==0]
#zero_a_zero_df.to_csv('zeroazero.csv')
df = df.drop(zero_a_zero_df.index)

Finalmente, vamos fazer a extração das 10 ações que tiveram maior e menor percentual de variação diária neste período.

In [ ]:
major_perc_df = df.nlargest(n=20, columns='perc_fechamento', keep='all')
minor_perc_df = df.nsmallest(n=20, columns='perc_fechamento', keep='all')
major_perc_df.to_csv('maiores_perc_var_072022.csv')
minor_perc_df.to_csv('menores_perc_var_072022.csv')


Obrigado e parabéns por ter chegado até aqui, espero que este artigo tenha trazido informações úteis para o seu aprendizado. Mas, o trabalho ainda não acabou. 
Vamos agora, construir um Dashboard no Power BI para mostrar nossas análises de forma clara e impactante.
Logo logo, farei um artigo onde irei mostrar como transformar os passos feitos hoje em funções, para serem aplicados nos mais diversos DataSets.